In [1]:
import uvicorn
from fastapi import FastAPI, APIRouter, HTTPException, Body
from typing import Type, Dict, Any, Callable
import inspect
from functools import wraps
from dslmodel import DSLModel, init_lm
from pydantic import Field
import nest_asyncio
import asyncio
from pyngrok import ngrok


In [2]:
nest_asyncio.apply()


In [6]:
def auto_router(model_class: Type[DSLModel]):
    router = APIRouter()
    model_name = model_class.__name__.lower()
    model_instance = model_class()

    @router.post(f"/{model_name}/")
    async def create_model_instance(data: Dict[str, Any]):
        init_lm()
        try:
            instance = model_class.from_prompt(**data)
            return instance.model_dump()
        except Exception as e:
            raise HTTPException(status_code=400, detail=str(e))

    return router


In [7]:
def create_endpoint(instance: DSLModel, method_name: str, method: Callable):
    sig = inspect.signature(method)

    @wraps(method)
    async def endpoint(data: Dict[str, Any] = Body(...)):
        init_lm()
        try:
            bound_args = sig.bind_partial(**data)
            bound_args.apply_defaults()
            result = method(instance, **bound_args.arguments)
            if asyncio.iscoroutine(result):
                result = await result
            if isinstance(result, DSLModel):
                return result.model_dump()
            return result
        except Exception as e:
            raise HTTPException(status_code=400, detail=str(e))

    return endpoint


In [8]:
@auto_router
class Participant(DSLModel):
    name: str = Field(..., description="Name of the participant.")
    role: str = Field(..., description="Role of the participant.")

    def greet(self) -> str:
        """Greet the participant."""
        return f"Hello, {self.name}! Your role is {self.role}."

    def update_role(self, new_role: str) -> Dict[str, Any]:
        """Update the participant's role."""
        self.role = new_role
        return {"message": f"Role updated to {self.role}"}


ValidationError: 2 validation errors for Participant
name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
role
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing